In [1]:
import os
import time
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

from aesvdd import autoencoder
from aesvdd import model
from aesvdd import datasets

import warnings
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

from tqdm import tqdm

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
print(os.getcwd())
warnings.filterwarnings("ignore")


Instructions for updating:
non-resource variables are not supported in the long term
/home/kael99/Documents/Code/Autoencoder_SVDD


In [2]:
X_train = np.fromfile("./data/X_train_100k.dat").reshape(-1, 9)
y_train = np.fromfile("./data/y_train_100k.dat", dtype=np.int64)
X_test = np.fromfile("./data/X_test_20k.dat").reshape(-1, 9)
y_test = np.fromfile("./data/y_test_20k.dat", dtype=np.int64)

In [3]:
lines = open("creat_checkpoints.csv")

In [4]:
name = 0
res_auc = []
for line in tqdm(lines):
    #setting
    setting = tuple(line.split())
    svdd_object, nu, lr, epochs = setting[0], float(setting[1]), float(setting[2]), int(setting[3])
    
    #creat autoencoder model
    tf.reset_default_graph()
    ae_model = autoencoder.VAE(z_dim=5)

    # pretrain
    ae_model.fit(X_train, shuffle=True, epochs=1, batch_size=100, validation_data=(X_test, None), verbose=0)

    inputs = ae_model.input
    outputs = ae_model.get_layer(name="code_layer").output
    encoder = Model(inputs=inputs, outputs=outputs)

    #creat autoencoder svdd
    ae_svdd = model.AESVDD(encoder, input_shape=(9,), representation_dim=5, objective=svdd_object, nu=nu, lr=lr)
    hisory = ae_svdd.fit(X_train, X_test, y_test, epochs=epochs, verbose=False)
    score = ae_svdd.predict(X_test)
    auc = roc_auc_score(y_test, -score)
    res_auc.append(line.strip("\n") + " " + str(auc) + "\n")
    ae_svdd.save_model(str(name))
    name += 1

lines.close()

0it [00:00, ?it/s]

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.

Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.


1it [00:03,  3.10s/it]


In [5]:
f = open("creat_checkpoints.csv", "w")
for line in res_auc:
    f.write(line)
f.close()